In [581]:
# Important libaries
import pandas as pd
import nltk
from nltk.tokenize import WhitespaceTokenizer as w_tokenizer
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer("english")

In [582]:
df = pd.read_table("/kaggle/input/LIARbyYang/train.tsv", header = None)
df.columns = ["ID", "label", "statement", "subject", "speaker", "speaker_job", "state", "party", "pof_count", "false_count", "barelytrue_count", "halftrue_count", "mostlytrue_count", "context"]

In [583]:
df.head(10)


,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602.json,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


In [584]:
df.shape

(10240, 14)

# Merge label

In [585]:
merger = { 'pants-fire' : 0,
           'false' : 0,
           'barely-true': 0,
           'half-true': 0,
           'mostly-true': 1,
           'true': 1}
df["label"].replace(merger, inplace=True)
df.head(10)

,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,0,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153.json,0,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602.json,0,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,1,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


# Clean data

In [608]:
# get rid of all periods
df['statement'].replace('\.','',regex=True,inplace=True) 
df['context'].replace('\.','',regex=True,inplace=True)

# get rid of all commas
df['statement'].replace(',','',regex=True,inplace=True) 
df['context'].replace(',','',regex=True,inplace=True) 

# clean quotations
df['statement'].replace('’','\'',regex=True,inplace=True) 
df['statement'].replace('‘','\'',regex=True,inplace=True) 
df['context'].replace('’','\'',regex=True,inplace=True) 
df['context'].replace('‘','\'',regex=True,inplace=True) 

df['statement'].replace('“','\"',regex=True,inplace=True) 
df['statement'].replace('”','\"',regex=True,inplace=True) 
df['statement'].replace('``','\"',regex=True,inplace=True) 
df['context'].replace('“','\"',regex=True,inplace=True) 
df['context'].replace('”','\"',regex=True,inplace=True) 
df['context'].replace('``','\"',regex=True,inplace=True) 

In [587]:
df.head(10)

,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech
2,324.json,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,0,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release
7,153.json,0,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a Democratic debate in Philadelphia Pa
8,5602.json,0,However it took $195 million in Oregon Lottery...,jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,1,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


# Bag of Words Process on Statements

In [588]:
df["statement_token"] = df["statement"].apply(nltk.word_tokenize) # tokenize statement

In [589]:
df['statement_token'].head(5)

0    [Says, the, Annies, List, political, group, su...
1    [When, did, the, decline, of, coal, start, ?, ...
2    [Hillary, Clinton, agrees, with, John, McCain,...
3    [Health, care, reform, legislation, is, likely...
4    [The, economic, turnaround, started, at, the, ...
Name: statement_token, dtype: object

In [590]:
df['stemmed_statement_token'] = df['statement_token'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

Vectorize tokens

In [591]:
df['stemmed_statement_token']

0        [say, the, anni, list, polit, group, support, ...
1        [when, did, the, declin, of, coal, start, ?, i...
2        [hillari, clinton, agre, with, john, mccain, `...
3        [health, care, reform, legisl, is, like, to, m...
4        [the, econom, turnaround, start, at, the, end,...
                               ...                        
10235    [there, are, a, larger, number, of, shark, att...
10236    [democrat, have, now, becom, the, parti, of, t...
10237    [say, an, altern, to, social, secur, that, ope...
10238    [on, lift, the, us, cuban, embargo, and, allow...
10239    [the, depart, of, veteran, affair, has, a, man...
Name: stemmed_statement_token, Length: 10240, dtype: object

See the most common (and potentially useless) words for the dataset

In [592]:
# bag of words for the whole dataset
def countwords(x):
    word2count = {}
    for words in x:
        for word in words:
            if word not in word2count.keys():
                word2count[word] = 1
            else:
                word2count[word] += 1
    # sort the word2count
    sorted_word2count = sorted(word2count.items(), key=lambda x:x[1], reverse = True)
    return sorted_word2count
            
words_in_statements = countwords(df["stemmed_statement_token"])




Check for redundant words:

In [593]:
words_in_statements[0:20]

[('the', 9783),
 ('in', 5115),
 ('of', 4806),
 ('to', 4478),
 ('a', 3672),
 ('and', 2822),
 ('say', 2504),
 ('for', 2117),
 ('that', 1965),
 ('is', 1802),
 ('on', 1516),
 ('state', 1372),
 ('have', 1320),
 ('has', 1301),
 ('$', 1285),
 ('percent', 1185),
 ('year', 1163),
 ('are', 1135),
 ('than', 1103),
 ('it', 1085)]

Apply CountVectorizer, ignore the first few rendundant words and words that appear only once.

In [609]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer=lambda x: x, max_df = 1400 ,min_df = 2) # need analyzer=lambda x: x to disable the analyzer, since the input is pandas series of lists.
X = vectorizer.fit_transform(df['stemmed_statement_token'])
new_features = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out()) # Get new features from countvectorizer
new_features.columns = 'statement_' + new_features.columns # add prefix to the column names


Merge new features

In [610]:
df2 = pd.concat([df,new_features],axis=1)

Do the same for context. But there are 102 null present, drop them first.

In [611]:
sum(df2['context'].isnull())

102

In [612]:
# drop rows with null context
df2.dropna(subset=['context'], inplace=True, ignore_index = True)

In [613]:
df2["context_token"] = df2["context"].apply(nltk.word_tokenize) # tokenize statement
df2['stemmed_context_token'] = df2['context_token'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

In [614]:
df2.shape

(10138, 5262)

Check redundant:

In [615]:
words_in_context = countwords(df2["stemmed_context_token"])
print(words_in_context[0:20])

[('a', 6635), ('an', 2098), ('the', 1840), ('interview', 1754), ('on', 1753), ('in', 1202), ('speech', 1062), ("''", 903), ('``', 902), ('ad', 877), ('news', 864), ('campaign', 807), ('debat', 767), ('press', 686), ('releas', 677), ("'s", 659), ('post', 492), ('with', 473), ('tv', 466), ('of', 463)]


In [621]:
len(words_in_context)

2085

Apply vectorizer:

In [623]:
cv = CountVectorizer(analyzer=lambda x: x, max_df = 1800 ,min_df = 4) # need analyzer=lambda x: x to disable the analyzer, since the input is pandas series of lists.
X2 = cv.fit_transform(df2['stemmed_context_token'])
new_features_context = pd.DataFrame(X2.toarray(), columns = cv.get_feature_names_out()) # Get new features from countvectorizer
new_features_context.columns = 'context_' + new_features_context.columns # add prefix to the column names


In [625]:
new_features_context

(10138, 2085)

In [626]:
df3 = pd.concat([df2,new_features_context],axis=1)

In [604]:
#df3.drop(columns=['statement', 'statement_token', 'stemmed_statement_token','context', 'context_token', 'stemmed_context_token'], inplace=True)

In [627]:
df3.shape

(10138, 7347)

# Test

Show unique value for "context"

Show all rows that contain: